# Exercise: Performance Optimization 2

Optimize the following function.

In [1]:
function work!(A, B, v, N)
    val = zero(eltype(v))
    for i in 1:N
        for j in 1:N
            val = mod(v[i],256)
            A[i,j] = B[i,j] * (sin(val) * sin(val) - cos(val) * cos(val))
        end
    end
    return A
end

work! (generic function with 1 method)

The following data is **fixed** and **not supposed to be modified**!

In [2]:
using Random
Random.seed!(42)

const N = 4000
const A = zeros(N,N)
const B = rand(N,N)
const v = rand(Int, N);

const A_result = work!(A,B,v,N);

You can compare against `A_result` to test your implementation(s):

In [3]:
using Test

@test work!(A,B,v,N) ≈ A_result

Test Passed

You can benchmark as follows:

In [4]:
using BenchmarkTools
@btime work!($A, $B, $v, $N); # or use @benchmark for more information

  836.749 ms (0 allocations: 0 bytes)


## Your Optimizations

Your optimized variants go here!

**Hints** (hopefully):
* What is suboptimal about the code? What is it that you'd want to change (but can't directly)?
* Sometimes writing the code in a different way doesn't give direct speedups but enables further optimization.
* A >30x speedup should be possible on Noctua 2 😉

In [ ]:
# Your code ...

In [11]:
function work_opt!(A, B, v, N)
    val = mod.(v,256)
    coef = (sin.(val).^2 .- cos.(val).^2)
    A .= B .* coef
    return A
end

work_opt! (generic function with 1 method)

In [12]:
@test work_opt!(A,B,v,N) ≈ A_result

Test Passed

In [13]:
@benchmark work_opt!(A,B,v,N)

BenchmarkTools.Trial: 211 samples with 1 evaluation.
 Range (min … max):  17.884 ms … 54.938 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.810 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.663 ms ±  4.662 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂ █▅ ▄▅▄      ▂   ▃                                        
  ▆▅████▄████▇▇▇▃██▆█▄█▆▇▆▅██▅▁▇▆▃▁▇▄▆▅▄▁▄▄▆▆▃▃▅▁▁▁▁▄▁▁▁▃▄▁▃▃ ▄
  17.9 ms         Histogram: frequency by time          35 ms <

 Memory estimate: 62.62 KiB, allocs estimate: 8.

## Bonus Question: Performance limit?

Look at your final optimized version of `work!`.

* What is conceptually limiting the performance, the compute capability or memory transfer?
* Assuming that a single CPU-core in Noctua 2 can achieve a **maximal memory bandwidth of ~45 GB/s**, can you give a performance bound estimate, i.e. the minimal runtime that we could possibly hope to achieve?
  * Hint: how many flops are performed per iteration and how many bytes are transferred?
* How far off is your implementation from achieving the limit (in percent)?